In [5]:
import tensorflow as tf
import numpy as np
from data_input import get_sparse_data
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.utils import to_categorical
from classifiers import check_acc
from keras import regularizers

[y_train,X_raw_train,X_train_sparse],[y_val,X_raw_val,X_val_sparse],[y_test,X_raw_test,X_test_sparse] = get_sparse_data(max_df= 0.996,min_df= 0.001)

#convert -1 into 2
y_train = [2 if x==-1 else x for x in y_train]
y_test = [2 if x==-1 else x for x in y_test]
y_val = [2 if x==-1 else x for x in y_val]
#convert into one hot
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_val_hot = to_categorical(y_val)

input_shape = X_train_sparse.get_shape()

In [19]:
reg = regularizers.l1(0)

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=input_shape[1],
                activity_regularizer= reg,
                kernel_regularizer=reg))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', input_dim=512,
                activity_regularizer= reg,
                kernel_regularizer=reg))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', input_dim=128,
                activity_regularizer= reg,
                kernel_regularizer=reg))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )


# Train the model, iterating on the data in batches of 32 samples
hist = model.fit(X_train_sparse.toarray(), y_train_hot, epochs=6,
          validation_data=(X_val_sparse.toarray(),y_val_hot),
          batch_size= 1024,shuffle = True
          ) 

Train on 14221 samples, validate on 7110 samples
Epoch 1/6
14221/14221 [==============================] - 1s - loss: 1.0899 - acc: 0.3969 - val_loss: 1.0792 - val_acc: 0.4135
Epoch 2/6
14221/14221 [==============================] - 0s - loss: 1.0723 - acc: 0.4103 - val_loss: 1.0593 - val_acc: 0.4135
Epoch 3/6
14221/14221 [==============================] - 0s - loss: 1.0315 - acc: 0.4106 - val_loss: 0.9957 - val_acc: 0.4204
Epoch 4/6
14221/14221 [==============================] - 0s - loss: 0.9398 - acc: 0.5285 - val_loss: 0.9170 - val_acc: 0.5994
Epoch 5/6
14221/14221 [==============================] - 0s - loss: 0.8400 - acc: 0.6449 - val_loss: 0.8756 - val_acc: 0.6100
Epoch 6/6
14221/14221 [==============================] - 0s - loss: 0.7565 - acc: 0.6769 - val_loss: 0.8730 - val_acc: 0.6121


In [21]:
classes = model.predict(X_test_sparse.toarray())
result = np.argmax(classes,axis=1)
acc = np.mean(result == y_test) *100.
print(acc)

60.5396290051
